In [1]:
import pandas as pd

In [2]:
df_read = pd.read_parquet("../data/interim/dfi_ensemble_2022-07-01_2025-10-01_V.1.0.parquet")

In [3]:
df_read.columns

Index(['reporting_month', 'ISRC', 'spotify', 'release_type', 'continent',
       'zone', 'quantity', 'unit_price', 'mechanical_fee', 'share_rate',
       'revenue'],
      dtype='object')

In [23]:
df_read.head()

,reporting_month,ISRC,spotify,release_type,continent,zone,quantity,unit_price,mechanical_fee,share_rate,revenue
0,2022/09/01,CA-5KR-00-21353,Other,Music Release,Europe,Italy,1,0.000007,0.0,0.765,0.000005
1,2022/09/01,GX-5MX-22-31727,Other,Music Release,Europe,Turkey,1,0.000007,0.0,0.765,0.000005
2,2022/09/01,GX-5MX-22-31730,Other,Music Release,LATAM,Uruguay,1,0.000010,0.0,0.765,0.000008
3,2022/09/01,CA-5KR-21-13899,Other,Music Release,Europe,Bosnia and herzegovi,5,0.000002,0.0,0.765,0.000008
4,2022/09/01,CA-5KR-21-77573,Other,Music Release,Asia,Jordan,1,0.000014,0.0,0.765,0.000011


In [19]:
df_filtered = df_read[
    (df_read['spotify'] == 'Spotify') & \
    (df_read['release_type'] == 'Music Release') & \
    (df_read['continent'] == 'Europe')
]

In [35]:
df_historic = (
    df_read
    .groupby(['ISRC'], as_index=False)
    .agg(
        revenue=('revenue', 'sum'),
        platform=('spotify','nunique'),
        historic_len=('reporting_month', 'nunique'),
        first_month=('reporting_month', 'min'),
        last_month=('reporting_month', 'max'),
        continents=('continent', 'nunique'),
        zones = ('zone', 'nunique')
    )
).sort_values(
    by=['historic_len', 'revenue', 'continents', 'zones'],
    ascending=[False, False, False, False]
)

# df_billing_artist_2['Billing'] = df_billing_artist_2['Billing'].round(2)

# df_billing_artist_2['billing_ratio'] = (
#     df_billing_artist_2['Billing'] / df_billing_artist_2['historic_len']
# )

# df_billing_artist_2 = df_billing_artist_2.sort_values(
#     by='billing_ratio',
#     ascending=False
# )

display(df_historic)

,ISRC,revenue,platform,historic_len,first_month,last_month,continents,zones
1832,GX-5MX-22-11692,9785.653631,2,38,2022/09/01,2025/10/01,7,160
1840,GX-5MX-22-90812,8469.657898,2,38,2022/09/01,2025/10/01,7,188
43,CA-5KR-21-33832,4083.549228,2,38,2022/09/01,2025/10/01,7,156
24,CA-5KR-21-13900,2986.162275,2,38,2022/09/01,2025/10/01,7,148
26,CA-5KR-21-25666,1759.908612,2,38,2022/09/01,2025/10/01,7,138
...,...,...,...,...,...,...,...,...
338,FR-26V-25-23421,0.000591,1,1,2025/09/01,2025/09/01,1,1
804,FR-S18-24-38769,0.000431,1,1,2024/11/01,2024/11/01,1,1
1293,FR-S18-24-40251,0.000415,1,1,2024/12/01,2024/12/01,1,1
1151,FR-S18-24-39787,0.000358,1,1,2025/04/01,2025/04/01,1,1


In [36]:
df_read['reporting_month'].max()

'2025/10/01'

In [37]:
df_toy = df_read[
    (df_read['reporting_month'] > '2023/10/01') & \
    (df_read['ISRC'].isin(['GX-5MX-22-11692', 'GX-5MX-22-90812', 'CA-5KR-21-33832']))
]

In [46]:
df_toy.to_parquet(
    "../data/interim/dfit_ensemble_2022-07-01_2025-10-01_V.1.0.parquet",
    engine="pyarrow",   # recomendado
    index=False
)